<a href="https://colab.research.google.com/github/stefanziog/Quantized-Deep-Neural-Networks/blob/main/tf_flower_dynamic_range.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)
import glob
import PIL
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from matplotlib import pyplot
import os
import numpy as np
import pathlib
import sys
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def load_dataset():
# load dataset
  dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
  data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
  data_dir = pathlib.Path(data_dir)

  image_count = len(list(data_dir.glob('*/*.jpg')))
  print(image_count)
  roses = list(data_dir.glob('roses/*'))
  PIL.Image.open(str(roses[0]))

  categories = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]



  daisy = list(data_dir.glob('daisy/*'))
  dandelion = list(data_dir.glob('dandelion/*'))
  roses = list(data_dir.glob('roses/*'))
  sunflowers = list(data_dir.glob('sunflowers/*'))
  tulips = list(data_dir.glob('turlips/*'))

  data = []
  labels = []

  for i in daisy:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size =  (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(0)
  for i in dandelion:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(1)
  for i in roses:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(2)
  for i in sunflowers:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(3)
  for i in tulips:
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
    target_size= (280,280))
    image=np.array(image)
    data.append(image)
    labels.append(3)
  data = np.array(data)
  labels = np.array(labels)

  from sklearn.model_selection import train_test_split
  train_images, test_images, train_labels, test_labels = train_test_split(data, labels, test_size=0.2,
                                                random_state=42)
	
  return train_images, train_labels, test_images, test_labels

In [ ]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [ ]:
#FOR THE QNN MODEL
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for test_image in test_images:
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == test_labels[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  return accuracy

In [ ]:
# load dataset
train_images, train_labels, test_images, test_labels = load_dataset()

# prepare pixel data
train_images, test_images = prep_pixels(train_images, test_images)

#load model
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Διπλωματική Κώδικες τελικοί/saved_models/tf_flower.h5')

228827136/228813984 [==============================] - 3s 0us/step
3670


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp55a4qk3f/assets


In [ ]:
tflite_models_dir = pathlib.Path("/tmp/flower_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
tflite_model_file = tflite_models_dir/"flower_model.tflite"
tflite_model_file.write_bytes(tflite_model)

10771216

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"flower_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpij_vrk6t/assets


INFO:tensorflow:Assets written to: /tmp/tmpij_vrk6t/assets


2714912

In [ ]:
ls -lh {tflite_models_dir}

total 13M
-rw-r--r-- 1 root root 2.6M Sep 17 08:38 flower_model_quant.tflite
-rw-r--r-- 1 root root  11M Sep 17 08:38 flower_model.tflite


In [ ]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [ ]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [ ]:
test_image = np.expand_dims(test_images[0], axis=0).astype(np.float32)

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

interpreter.set_tensor(input_index, test_image)
interpreter.invoke()
predictions = interpreter.get_tensor(output_index)

In [ ]:
print(evaluate_model(interpreter))
print(evaluate_model(interpreter_quant))

0.768695652173913
0.7669565217391304
